In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 46.7 MB/s 
     |████████████████████████████████| 9.3 MB 22.4 MB/s 
     |████████████████████████████████| 108 kB 56.6 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [4]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [5]:
df.drop(['packer_type'],axis=1,inplace=True)

In [6]:
from sklearn.utils import shuffle
df = shuffle(df)

In [7]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [8]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [9]:
df.drop(['index'],axis=1,inplace=True)

In [10]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,208,4,1,0,1,...,1,1,3,0,0.000000,5.564097,39424,5.584758,0,1
1,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,5.541458,0.299051,8704,4.778737,1,0
2,80,2,4,65535,184,256,10,1,0,1,...,1,6,4,1,0.000000,0.000000,39946,7.411158,1,1
3,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,6.586567,2.885577,104448,7.461375,0,1
4,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.239523,0.000000,12416,3.277502,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,7.686025,7.925198,842240,7.904670,1,1
5206,144,3,4,65535,184,176,3,1,1,1,...,1,2,1,1,0.000000,0.000000,36352,7.718656,1,1
5207,144,3,4,65535,184,224,7,1,1,1,...,1,3,4,0,6.174061,1.014414,25600,5.433165,0,1
5208,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.752945,7.532575,120320,7.020006,0,1


In [11]:
df_5percent=df[0:4689]

In [12]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,208,4,1,0,1,...,1,1,3,0,0.000000,5.564097,39424,5.584758,0,1
1,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,5.541458,0.299051,8704,4.778737,1,0
2,80,2,4,65535,184,256,10,1,0,1,...,1,6,4,1,0.000000,0.000000,39946,7.411158,1,1
3,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,6.586567,2.885577,104448,7.461375,0,1
4,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.239523,0.000000,12416,3.277502,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4684,144,3,4,65535,184,248,5,1,0,1,...,1,1,4,0,6.195667,1.096181,19968,5.852790,1,0
4685,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,6.120545,4.733026,38168,6.532960,1,0
4686,144,3,4,65535,184,128,6,1,0,1,...,1,1,5,0,6.632689,1.913984,260096,6.656836,0,0
4687,144,3,4,65535,184,232,6,1,1,1,...,1,1,5,0,7.092910,7.511052,573440,7.814586,1,1


In [13]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2253, 69)
class 1: (2436, 69)


In [14]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [15]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     208.0               4.0   
1      144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
2       80.0   2.0        4.0     65535.0  184.0     256.0              10.0   
3      144.0   3.0        4.0     65535.0  184.0     216.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     192.0               2.0   
...      ...   ...        ...         ...    ...       ...               ...   
4684   144.0   3.0        4.0     65535.0  184.0     248.0               5.0   
4685   144.0   3.0        4.0     65535.0  184.0     264.0               5.0   
4686   144.0   3.0        4.0     65535.0  184.0     128.0               6.0   
4687   144.0   3.0        4.0     65535.0  184.0     232.0               6.0   
4688   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   

      C

In [16]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4689.0,...,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4689.000000,4.689000e+03,4689.000000,4689.000000,4689.000000
mean,154.217957,11.492216,9.449350,65107.441032,190.622308,222.774152,4.650459,0.989977,0.366816,1.0,...,0.998934,1.344423,3.306035,0.156537,4.956728,2.529802,8.152019e+05,6.368513,0.544679,0.519514
std,649.806047,413.985143,265.296593,5188.180003,363.102928,48.844694,1.914006,0.099625,0.481987,0.0,...,0.032641,1.622997,1.136187,0.363402,2.510522,2.692007,5.478647e+06,1.119448,0.498053,0.499672
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942085,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.894021,0.000000,6.144000e+04,5.696229,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.152807,1.573551,1.228800e+05,6.397076,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.509409,4.686923,3.087360e+05,7.318898,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,17744.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [17]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [18]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [19]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [20]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      53.125
0.0      46.875
dtype: float64

In [21]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [22]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [23]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [24]:
circuit(weights_init, data[0])

tensor(0.10100913, requires_grad=True)

In [25]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [26]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [27]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [28]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [29]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.22418116, requires_grad=False), tensor(-0.30898036, requires_grad=False)], Y =  1
X = [tensor(0.90565626, requires_grad=False), tensor(0.05804934, requires_grad=False)], Y =  1
X = [tensor(-0.26685491, requires_grad=False), tensor(-0.1466486, requires_grad=False)], Y = -1
X = [tensor(-0.2268037, requires_grad=False), tensor(-0.29932896, requires_grad=False)], Y =  1
X = [tensor(-0.2243034, requires_grad=False), tensor(-0.30801559, requires_grad=False)], Y = -1


In [30]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [31]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )


New best
Iter:     1 | Cost: 1.0830212 | Accuracy: 0.5375000 
New best
Iter:     2 | Cost: 1.0080493 | Accuracy: 0.5437500 
Iter:     3 | Cost: 0.9961643 | Accuracy: 0.5312500 
Iter:     4 | Cost: 0.9866041 | Accuracy: 0.5437500 
Iter:     5 | Cost: 0.9816224 | Accuracy: 0.5187500 
Iter:     6 | Cost: 0.9952690 | Accuracy: 0.5125000 
New best
Iter:     7 | Cost: 0.9536782 | Accuracy: 0.5500000 
New best
Iter:     8 | Cost: 0.9436054 | Accuracy: 0.5687500 
Iter:     9 | Cost: 0.9612396 | Accuracy: 0.5687500 
Iter:    10 | Cost: 0.9658458 | Accuracy: 0.5687500 
New best
Iter:    11 | Cost: 0.9096512 | Accuracy: 0.5875000 
Iter:    12 | Cost: 0.9087179 | Accuracy: 0.5687500 
Iter:    13 | Cost: 0.9215884 | Accuracy: 0.5625000 
Iter:    14 | Cost: 0.9871217 | Accuracy: 0.5562500 
Iter:    15 | Cost: 1.0389580 | Accuracy: 0.5500000 
Iter:    16 | Cost: 1.0851304 | Accuracy: 0.5375000 
Iter:    17 | Cost: 1.0640309 | Accuracy: 0.5437500 
Iter:    18 | Cost: 0.9638591 | Accuracy: 0.5625000 
I

In [32]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [33]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.0962155256373025, Accuracy: 48.0%


In [34]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,-1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,-1.0
5,1.0,1.0
6,1.0,-1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,1.0


In [35]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         2
         1.0       0.90      0.50      0.64        38

    accuracy                           0.48        40
   macro avg       0.45      0.25      0.32        40
weighted avg       0.86      0.47      0.61        40

0.9047619047619048
0.5
0.6440677966101696
0.25
